A remake of the bar-and-whisker plot of statistical significance for the RDA paper based on RDA_bars_with_whiskers, but rewritten in much better style and to take advantage of more efficient loading of data.

In [1]:
from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from shutil import copy

In [2]:
data_file = 'Data/decadal_stats_tau.hdf'
results_df = pd.read_hdf(data_file)
results_df = results_df.sort_index().sort_index(axis=1)

In [3]:
results_df

Period                      all                    fall              \
Years                 8007_5179    9407_8093  8007_5179   9407_8093   
Metric    Stat                                                        
frequency diff_p2p1 -0.00489237  0.000782779  0.0138612 -0.00759878   
          mean_p1      0.386301     0.381018   0.260455    0.278116   
          mean_p2      0.381409       0.3818   0.274316    0.270517   
          pval         0.303991     0.522966    0.71668    0.413541   
          std_p1     0.00669284   0.00960763  0.0168119   0.0247029   
          std_p2     0.00679496   0.00961141  0.0173935    0.024491   
intensity diff_p2p1     0.36322     0.674041   0.753374     1.39691   
          mean_p1       23.2812       23.307    20.2594     20.3311   
          mean_p2       23.6445       23.981    21.0128      21.728   
          pval         0.945305     0.980302   0.853715    0.908009   
          std_p1       0.155693      0.21969   0.463724    0.691498   
          std_p2       0.163406     0.241994   0.510015    0.794565   
latitude  diff_p2p1   -0.249351    -0.163582   0.233537     0.74348   
          mean_p1       28.7349      28.5675    29.0985     28.9692   
          mean_p2       28.4856      28.4039     29.332     29.7127   
          pval       0.00309969     0.106289   0.784422    0.972503   
          std_p1      0.0650961      0.09196   0.208553     0.25641   
          std_p2      0.0637926    0.0904314   0.196724    0.293003   

Period                   meiyu               post-meiyu             pre-meiyu  \
Years                8007_5179   9407_8093    8007_5179  9407_8093  8007_5179   
Metric    Stat                                                                  
frequency diff_p2p1 -0.0243227 -0.00357143 -0.000607328  0.0136986 -0.0601293   
          mean_p1     0.668966    0.646429     0.421351   0.413894   0.591379   
          mean_p2     0.644643    0.642857     0.420744   0.427593    0.53125   
          pval        0.193552    0.464823     0.488811   0.671326    0.02024   
          std_p1       0.01954   0.0285706    0.0151769  0.0217882  0.0204117   
          std_p2     0.0202254   0.0286351    0.0154425  0.0218855  0.0210875   
intensity diff_p2p1   0.315293     2.66856     0.184915  -0.410096   0.362703   
          mean_p1      28.1122     27.0932       25.459    25.8524     25.392   
          mean_p2      28.4275     29.7618      25.6439    25.4423    25.7547   
          pval        0.714829      0.9997     0.647035   0.277372   0.756724   
          std_p1      0.378743     0.50701     0.348549   0.495789   0.354868   
          std_p2      0.388765     0.58523     0.345229   0.482455   0.397652   
latitude  diff_p2p1  -0.178086    -1.09609    -0.605018  -0.390166   -0.14285   
          mean_p1      29.6035     29.9735      30.1928    29.7861    27.4985   
          mean_p2      29.4254     28.8774      29.5878     29.396    27.3556   
          pval        0.205879  0.00029997   0.00489951   0.111589   0.241276   
          std_p1      0.157057     0.21946     0.168509   0.230248   0.132586   
          std_p2      0.154459    0.214518     0.164343   0.228764   0.154209   

Period                              spring             
Years                 9407_8093  8007_5179  9407_8093  
Metric    Stat                                         
frequency diff_p2p1  0.00178571  0.0119317 -0.0433255  
          mean_p1      0.530357   0.465235   0.498829  
          mean_p2      0.532143   0.477166   0.455504  
          pval         0.516886   0.690855   0.102287  
          std_p1      0.0298256  0.0167714  0.0241966  
          std_p2      0.0298189  0.0170918  0.0241007  
intensity diff_p2p1     1.32048   0.828424  0.0227796  
          mean_p1       25.0903    19.7509    20.5685  
          mean_p2       26.4108    20.5794    20.5913  
          pval         0.952405   0.984302   0.516348  
          std_p1       0.515294   0.250295   0.384375  
          std_p2      

In [4]:
#customized script that adds overbars to indicate time periods, and then on top of that a p-value for the significance of the change.
def add_pvals(fig,ax,rects_in,errs_list,pvals_list):
    
    #reorders rectangles so that we're dealing with group of 3 rectangles for each time period at same time
    zipped_rects = zip(rects_in[0],rects_in[1],rects_in[2])
    
    #get y-axis limits
    yax = ax.get_ylim()
    yhgt = yax[1]-yax[0]
    
    #add p-values sequentially to each group of 3 columns
    for rects,errs,pvals in zip(zipped_rects,errs_list,pvals_list):
        
        ##each rects group contains 3 bars - 1st bar is for '51-'79 and is wider, 2nd is '80-'93 and 3rd is '94-'07
        heights = [ rect.get_height() + err for rect,err in zip(rects,errs) ]
        max_height = max(heights[0],heights[1],heights[2])

        ## figure out x and y coordinates of brackets that are placed above bars
        xx_l = [rects[0].get_x() + rects[0].get_width()/2, rects[1].get_x() + rects[1].get_width()/2]
        xx_r = [ xx_l[0] + rects[0].get_width(), xx_l[1] + rects[1].get_width() ]
        yy = [max_height + .01*yhgt, max_height + .13*yhgt]
        bracket_widths = [1.25,.625]
        
        for x_l, x_r, y, bwidth, pval in zip(xx_l, xx_r, yy, bracket_widths, pvals):
            ##add brackets
            ax.annotate("", xy=(x_l, y + .015*yhgt), xytext=(x_l, y + .025*yhgt), 
                fontsize=15, ha='center', va='bottom',
                arrowprops=dict(arrowstyle='-[, widthB='+str(bwidth)+', lengthB=.5', lw=1.0))

            ax.annotate("", xy=(x_r, y + .015*yhgt), xytext=(x_r, y + .025*yhgt), 
                fontsize=15, ha='center', va='bottom',
                arrowprops=dict(arrowstyle='-[, widthB='+str(bwidth)+', lengthB=.5', lw=1.0))

            #add extra line
            ax.plot([x_l, (x_l+x_r)/2], [y + .022*yhgt, y + .045*yhgt], 'k-', lw=1)
            ax.plot([x_r, (x_l+x_r)/2], [y + .022*yhgt, y + .045*yhgt], 'k-', lw=1)

            #add p-value
            font = FontProperties()
            font.set_size('large')
            
            if abs(pval-.5) > .475:
                font.set_weight('bold')
                if abs(pval-.5) > .495:
                    font.set_style('italic')
            if pval > .999:
                message = 'p>.999'
            elif pval < .001:
                message = 'p<.001'
            else:
                message = "p={0:.3f}".format(round(pval,3))

            ax.text((x_l+x_r)/2, y + .06*yhgt, message, horizontalalignment = 'center', fontproperties = font)

In [5]:
N = 6               # number of data entries
ind = np.arange(N)              # the x locations for the groups
width1 = 0.35                   # bar width
width2 = 0.175

In [6]:
fig, axes = plt.subplots(3,1,figsize=(12,18), facecolor='w')
rects = []

## Parameters for subplots
colors = ['MediumSlateBlue','Tomato','Gold']
metrics = ['frequency','latitude','intensity']
my_error_kw={'ecolor':'Black', 'linewidth':1, 'capsize':2}
periods = ['p1','p1','p2']
seasons = ['spring','pre-meiyu','meiyu','post-meiyu','fall','all']
subplot_titles = ['a) Frontal Rain Event (FRE) Frequency (%)','b) Mean FRE Latitude ($^{\circ}$N)',\
                  'c) Mean FRE Daily Accumulation (mm day$^{-1}$)']
widths = [width1, width2, width2]
xpositions = [ind-(width1/2),ind+(width2/2),ind+(3*width2/2)]
xtick_labels = ('Spring', 'Pre-Meiyu', 'Meiyu', 'Post-Meiyu', 'Fall','Full Year')
yaxis_labels = ['Frequency (%)','Latitude ($^{\circ}$N)','Daily Accumulation (mm day$^{-1}$)']
year_tags = ['8007_5179','9407_8093','9407_8093']
yranges = [[0,.9],[26.8,31.6],[19,34]]


for ax,met,title,ylabel,yrange in zip(axes,metrics,subplot_titles,yaxis_labels,yranges):
    
    #list of all 3 bar plots for a given subpanel
    sub_rects = []
    
    ## Plot set of bars for each time period
    ## the [seasons] toggle organizes values in correct order
    for (color, p, width, xpos, yr_tag) in zip(colors, periods, widths, xpositions, year_tags):
        vals = results_df.loc[(met,'mean_'+p),(slice(None),yr_tag)][seasons].values
        errs = results_df.loc[(met,'std_'+p),(slice(None),yr_tag)][seasons].values
        sub_rects.append(ax.bar(xpos,vals,width, color=color, yerr=errs, error_kw=my_error_kw,\
                               edgecolor='black'))
    
    ## Tweak plot formatting
    ax.legend((sub_rects[0], sub_rects[1], sub_rects[2]), ("'51-'79", "'80-'93", "'94-'07"),\
              prop={'size': 14})
    ax.set_title(title, fontsize=22)
    ax.set_xlim([-.5,5.5])
    ax.set_xticks(xpositions[0]+width1/2)
    ax.set_xticklabels(xtick_labels,\
                        fontsize=16)
    #ax.set_ylabel(ylabel, fontsize=16)
    ax.set_ylim([yrange[0],yrange[1]])

    ## Turn off x-axis ticks
    ax.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off')         # ticks along the top edge are off
    
    ## Change default size of y-axis tick labels
    ax.tick_params(
    axis='y',          # changes apply to the x-axis
    labelsize=14)         # ticks along the top edge are off
    
    rects.append(sub_rects)
    
    ## collect the right collection of errors and p-values necessary for plotting.
    sub_errs = []
    sub_pvals = []
    
    for season in seasons:
        
        ## arrange errors
        bar_errs = []
        for (p, yr_tag) in zip(periods,year_tags):
            bar_errs.append(results_df.loc[(met,'std_'+p),(season,yr_tag)])
        
        sub_errs.append(bar_errs)
        sub_pvals.append(list(results_df.loc[(met,'pval'),(season,slice(None))].values))

    #add p-values to top of plots
    add_pvals(fig,ax,sub_rects,sub_errs,sub_pvals)
  
## Change y-axis labels from frequency to percentage in subplot a
axes[0].set_yticklabels(np.linspace(0,90,10).astype(int))

#plt.tight_layout()

In [7]:
## SAVE FIGURE TO MANUSCRIPT AND ANALYSIS DIRECTORIES
figure_name = "bars.pdf"

savepath = "/Users/Siwen/Desktop/RDA/Figures/" + figure_name

if os.path.exists(savepath):
    print("Overwriting existing version of file in RDA manuscript directory...")
    os.remove(savepath)
    
plt.savefig(savepath, bbox_inches='tight')
    
copypath = "/Users/Siwen/RDA/Figures/" + figure_name
    
if os.path.exists(copypath):
    print("Overwriting existing version of file in RDA Python directory...")
    os.remove(copypath)
    
copy(savepath,copypath)

plt.savefig(savepath, bbox_inches='tight')

Overwriting existing version of file in RDA manuscript directory...
Overwriting existing version of file in RDA Python directory...
